In [1]:
PATH ="C:\\Users\\GANESH\\Desktop\\pyspark\\dataset2\\"
FILE = "marketing_sample_for_careerbuilder_usa-careerbuilder_job_listing__20200401_20200630__30k_data.ldjson"

#### Dendencies

In [2]:
import pyspark
from pyspark.sql import SparkSession as session
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas
pandas.set_option("display.max.column",None)
import re

#### Create Spark Session

In [3]:
spark = session.builder.appName("job").getOrCreate()
spark_df=spark.read.json(PATH+FILE,primitivesAsString = 'true')
print("Spark Session Created Successfully")

Spark Session Created Successfully


In [4]:
spark_df.printSchema()

root
 |-- city: string (nullable = true)
 |-- company_description: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- contact_email: string (nullable = true)
 |-- country: string (nullable = true)
 |-- crawl_timestamp: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- duplicate_status: string (nullable = true)
 |-- fitness_score: string (nullable = true)
 |-- geo: string (nullable = true)
 |-- has_expired: string (nullable = true)
 |-- html_job_description: string (nullable = true)
 |-- inferred_city: string (nullable = true)
 |-- inferred_country: string (nullable = true)
 |-- inferred_iso2_lang_code: string (nullable = true)
 |-- inferred_iso3_lang_code: string (nullable = true)
 |-- inferred_salary_currency: string (nullable = true)
 |-- inferred_salary_from: string (nullable = true)
 |-- inferred_salary_time_unit: string (nullable = true)
 |-- inferred_salary_to: string (nullable = true)
 |-- inferred_state: string (nullable = true)
 |-- is_

#### Drop duplicates and Unused Columns

In [5]:
spark_df =spark_df.drop('inferred_city','inferred_iso3_lang_code','inferred_salary_time_unit',\
                        'inferred_iso2_lang_code','inferred_state','last_expiry_check_date',\
                        'postdate_yyyymmdd')

In [6]:
def dropnul(spark_df):
    null = spark_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in spark_df.columns]).\
          collect()[0].asDict()
    col_to_drop = [key for key, value in null.items() if value > 0.7]
    spark_df = spark_df.drop(*col_to_drop)
    return spark_df
dropnul(spark_df)
new_spark_df = spark_df.transform(dropnul)
new_spark_df.show(10)

+--------------------+--------------------+--------------------+-----------------+----------------+-------------+---+-----------+--------------------+----------------+-------------+--------------------+-------------+--------------------+------------------------+----------+----------------------------+--------------+--------------------+-----+--------------------+--------------------+
|                city|        company_name|     crawl_timestamp|           domain|duplicate_status|fitness_score|geo|has_expired|html_job_description|inferred_country|    job_board|     job_description|job_post_lang|           job_title|latest_expiry_check_date| post_date|postdate_in_indexname_format|salary_offered|           site_name|state|             uniq_id|                 url|
+--------------------+--------------------+--------------------+-----------------+----------------+-------------+---+-----------+--------------------+----------------+-------------+--------------------+-------------+----------

calculate number of jobs posted on daily basis, per each city

In [7]:
jobs=spark_df.groupBy(["city"]).agg(
    count("post_date").alias("total_jobs_post"))
jobs = jobs.sort(desc("total_jobs_post")) 

# Convert to Pandas DataFrame and Save to Local
josbs_pd_df =jobs.toPandas()
josbs_pd_df.to_csv(PATH+'job_posts_data.csv',index=False)

In [8]:
print("Number of jobs Posts by cities on Daily Basis")
jobs.show(10)

Number of jobs Posts by cities on Daily Basis
+------------+---------------+
|        city|total_jobs_post|
+------------+---------------+
|      Boston|            262|
|     Chicago|            218|
|   Charlotte|            216|
|  Cincinnati|            212|
|    Portland|            212|
|  Louisville|            208|
|Philadelphia|            185|
|    New York|            182|
|     Atlanta|            177|
|      Denver|            169|
+------------+---------------+
only showing top 10 rows



calculate average salary per job title and state

In [9]:
sal =new_spark_df.select( 'job_title','state',split("salary_offered","[$/-]").getItem(1).cast('double').alias('salary'))
sal=sal.na.drop()
salary_df =sal.groupBy('job_title','state').agg(avg('salary').alias('salary in $/hour'),count('*').alias('count'))

In [10]:
# Convert to Pandas DataFrame and Save to Local
avg_salary = salary_df.na.drop().toPandas()
avg_salary.to_csv(PATH+"avg_salary.csv", index=False)

In [11]:
print("Average salary on the basis of Job title and State")
salary_df.show(10)

Average salary on the basis of Job title and State
+--------------------+-----+----------------+-----+
|           job_title|state|salary in $/hour|count|
+--------------------+-----+----------------+-----+
|      Sr. Accountant|   TX|            30.0|    1|
|Animal Care Tecni...|   NC|            15.0|    1|
|Traveling Heavy N...|   TX|            14.0|    1|
|Billing/ Accounti...|   MN|            20.0|    1|
|    Rebar Fabricator|   OH|            15.0|    1|
|    Assembler (T518)|   MO|           11.25|    1|
|Mechanical Assembler|   TN|            12.0|    1|
|           2nd shift|   MD|            13.1|    1|
|Customer Service ...|   TN|            11.5|    1|
|Manual Machinist ...|   WA|            18.0|    1|
+--------------------+-----+----------------+-----+
only showing top 10 rows



Identify the top 10 most active companies by number of positions opened

In [12]:
company=spark_df.groupBy(["company_name"]).agg(count("post_date").alias("number_of_position"))
company =company.sort(desc("number_of_position")).limit(10)

# Convert to Pandas DataFrame and Save to Local
company_pd_df=company.toPandas()
company_pd_df.to_csv(PATH+'Top_10_companies.csv',index=False)

In [13]:
print("Top 10 Most Active Companies")
company.show()

Top 10 Most Active Companies
+--------------------+------------------+
|        company_name|number_of_position|
+--------------------+------------------+
|  Amazon Fulfillment|              7501|
|                GPAC|               924|
|LanguageLine Solu...|               871|
|Lowe's Home Impro...|               861|
|      CDL Career Now|               545|
|  Advance Auto Parts|               492|
|             Aerotek|               477|
|         Robert Half|               444|
|    Driveline Retail|               372|
|            Circle K|               334|
+--------------------+------------------+



Create a UDF Function

In [14]:
#create a UDF function to clean job description from HTML code contained inside
# analyze and comment on the performance of the UDF
# function. How the performance could be improved
import time
start_time = time.time()

clean_html =udf(lambda row: re.sub(re.compile('<.*?>'), '',row))
new_spark_df =new_spark_df.withColumn('html_job_description', clean_html("html_job_description"))
print(f"Execution time: {time.time() - start_time}")

# Using split() function cleanup HTML
df2 = new_spark_df.select(split(col("html_job_description"),"Job Description ",2)\
                          .getItem(1).alias("new_job_description")) \
                           .drop("html_job_description")

Execution time: 0.05199766159057617


In [15]:
spark.stop()
print("SparkSession stopped")

SparkSession stopped
